# Teste com o dataset classificado pelo BERT

In [1]:
import pandas as pd
import numpy as np
import spacy
import os
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from spacy.cli.download import download

In [2]:
def criar_embeddings(df_tweets):
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets.Texto])

    return vetores


def ler_modelo(path: str):
    return pickle.load(open(path, 'rb'))


def salvar_modelo(path: str, modelo):
    return pickle.dump(modelo, open(path, 'wb'))    


def fazer_amostragem(train_dataset: pd.DataFrame):
    """

    :param train_dataset:
    :return:
    """
    sentimentos = train_dataset['Sentimento'].unique()
    df = pd.DataFrame([])

    for sentimento in sentimentos:
        df_filtrado = train_dataset.loc[train_dataset['Sentimento'] == sentimento][:600]
        df = pd.concat([df, df_filtrado])

    return df

In [3]:
path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman_2.csv'
).dropna()

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

In [4]:
# processamento dos dados para word embeddings
path_embeddings_treinamento = '../resources/modelos/embeddings_treinamento_novo.pkl'
path_embeddings_teste = '../resources/modelos/embeddings_teste_novo.pkl'

if os.path.exists(path_embeddings_treinamento):
    embeddings = ler_modelo(path_embeddings_treinamento)
else:
    embeddings = criar_embeddings(x_train)
    salvar_modelo(path_embeddings_treinamento, embeddings)

if os.path.exists(path_embeddings_teste):
    embeddings_teste = ler_modelo(path_embeddings_teste)
else:
    embeddings_teste = criar_embeddings(x_test)
    salvar_modelo(path_embeddings_teste, embeddings_teste)

Fazendo os word embeddings
Fazendo os word embeddings


In [5]:
path_svc = '../resources/modelos/svc.pkl'
path_lgr = '../resources/modelos/logistic_regression.pkl'
path_forest = '../resources/modelos/forest.pkl'

if os.path.exists(path_svc):
    svc = ler_modelo(path_svc)
else:
    svc = LinearSVC(C=100, random_state=0, dual=True, max_iter=10000)
    svc.fit(embeddings, y_train)
    salvar_modelo(path_svc, svc)
    
if os.path.exists(path_lgr):
    lgr = ler_modelo(path_lgr)
else:
    lgr = LogisticRegression(random_state=0, max_iter=10000)
    lgr.fit(embeddings, y_train)
    salvar_modelo(path_lgr, lgr)

if os.path.exists(path_forest):
    forest = ler_modelo(path_forest)
else:
    forest = RandomForestClassifier(random_state=0, n_jobs=-1)
    forest.fit(embeddings, y_train)
    salvar_modelo(path_forest, forest)

c:\Users\sefaz\Documents\Projetos Python\Analise-de-sentimentos-pandemia-covid19\tweets_venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [6]:
# previsões
previsoes_svc = svc.predict(embeddings_teste)
previsoes_lgr = lgr.predict(embeddings_teste)
previsoes_forest = forest.predict(embeddings_teste)

In [7]:
# testar performance
print('Relatório Linear SVC')
print(classification_report(y_test, previsoes_svc))

Relatório Linear SVC
              precision    recall  f1-score   support

       feliz       0.59      0.65      0.62        92
        medo       0.50      0.69      0.58        51
        nojo       0.84      0.61      0.71       220
       raiva       0.58      0.60      0.59        58
      triste       0.33      0.48      0.39        60

    accuracy                           0.61       481
   macro avg       0.57      0.61      0.58       481
weighted avg       0.66      0.61      0.62       481



In [8]:
print('Relatório Logistic Regression')
print(classification_report(y_test, previsoes_lgr))

Relatório Logistic Regression
              precision    recall  f1-score   support

       feliz       0.72      0.77      0.75        92
        medo       0.71      0.69      0.70        51
        nojo       0.83      0.78      0.80       220
       raiva       0.59      0.71      0.65        58
      triste       0.47      0.47      0.47        60

    accuracy                           0.72       481
   macro avg       0.67      0.68      0.67       481
weighted avg       0.73      0.72      0.72       481



In [9]:
print('Relatório Random Forest')
print(classification_report(y_test, previsoes_forest))

Relatório Random Forest
              precision    recall  f1-score   support

       feliz       0.64      0.55      0.59        92
        medo       0.00      0.00      0.00        51
        nojo       0.55      0.93      0.69       220
       raiva       0.42      0.09      0.14        58
      triste       0.31      0.08      0.13        60

    accuracy                           0.55       481
   macro avg       0.38      0.33      0.31       481
weighted avg       0.46      0.55      0.46       481



In [10]:
pontuacoes_logreg = cross_val_score(lgr, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted')
pontuacoes_svc = cross_val_score(svc, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted')
pontuacoes_forest = cross_val_score(forest, embeddings_teste, y_test, cv=5, n_jobs=-1, scoring='f1_weighted');

In [11]:
def exibir_pontuacoes(pontuacoes):
    soma_ponuacoes = 0

    for valor in pontuacoes:
        soma_ponuacoes += valor
        
    media = soma_ponuacoes / len(pontuacoes)
    
    print(f'Lista de pontuações: {pontuacoes}\nMédia: {media}' )
    
exibir_pontuacoes(pontuacoes_logreg)
exibir_pontuacoes(pontuacoes_svc)
exibir_pontuacoes(pontuacoes_forest)

Lista de pontuações: [0.61575121 0.64267069 0.60911738 0.63508692 0.64671752]
Média: 0.6298687437279203
Lista de pontuações: [0.57669679 0.63381704 0.60378251 0.60058661 0.58953032]
Média: 0.6008826524950031
Lista de pontuações: [0.35604396 0.46130152 0.43689227 0.40534705 0.50255284]
Média: 0.4324275281347279


In [1]:
import pickle
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman_2.csv'
).dropna()

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

In [2]:
import re 


def carregar_modelo():
    path_lgr = '../resources/modelos/logistic_regression.pkl'
    path_embeddings_treinamento = '../resources/modelos/embeddings_treinamento_novo.pkl'
    path_embeddings_teste = '../resources/modelos/embeddings_teste_novo.pkl'

    embeddings = pickle.load(open(path_embeddings_treinamento, 'rb'))
    embeddings_teste = pickle.load(open(path_embeddings_teste, 'rb'))
    lgr = pickle.load(open(path_lgr, 'rb'))

    return lgr


def limpar_texto(texto: str):
        texto = texto.lower()
        texto = re.sub('\n', '', texto)
        texto = re.sub('@\S+', '', texto)
        texto = re.sub(r'https://\S+', '', texto)

        return texto


def classificar(texto: str):
    nlp = spacy.load('pt_core_news_lg')

    modelo = carregar_modelo()
    texto = limpar_texto(texto)

    try:
        # tokenizamos a frase, vetorizamos e colocamos em uma array
        vetor = np.array(nlp(texto).vector)
        # colocamos o vetor no formato 1x300, que é o que o modelo espera
        vetor = np.reshape(vetor, (1, 300))
    except TypeError:
        texto = str(texto)
        vetor = np.array(nlp(texto).vector)
        vetor = np.reshape(vetor, (1, 300))

    # fazemos a previsão e retornamos a classe
    classes = modelo.classes_
    probabilidades = modelo.predict_proba(vetor)

    classes_probabilidades = dict({
        'classe': list(classes),
        'probabilidade': probabilidades.tolist()[0]
    })

    return classes_probabilidades

In [61]:
classificar('Eu tenho q estar grávida p realizarem meus desejos? Só pq eu não estou grávida?')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [0.1221697759570843,
  0.03782998378601986,
  0.00496657832294531,
  0.8067022114323416,
  0.028331450501608905]}

In [62]:
classificar('Um relógio de 200 anos, que sobreviveu a monarquia, as oligarquias, ao Varguismo, a ditadura, mas não sobreviveu ao #bolsonarismo, destruíram um #patrimônio inestimável pro país')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [0.08405164258977785,
  0.0012723040206095429,
  0.6822136784088942,
  0.00012894465651075164,
  0.23233343032420772]}

In [63]:
classificar('Parece q namoro um caminhoneiro um carteiro viajante um representante de vendas passa uma semana comigo e outras tantas viajando pra la e pra ca')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [0.03711378336027173,
  0.0005943812155693479,
  0.07439056357766399,
  0.22921234278030733,
  0.6586889290661876]}

In [64]:
classificar('Não é  hora de sensacionalismo, é  hora de unirem forças e  ajudar o povo é não fazer política.  Qnd pararem de querer ganhar em.cima do povo, td pode melhorar. ')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [0.0002929741663926448,
  0.008881476520543741,
  0.9401516796761431,
  0.0040391538698429244,
  0.046634715767077645]}

In [65]:
classificar('Acabo de cair num golpe terrível, lamentavelmente abri um pote de sorvete que tava no congelador, e era feijão')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [6.244690076693911e-05,
  7.965117880749732e-05,
  0.5078697012188962,
  0.0003546328294471549,
  0.4916335678720822]}

In [67]:
classificar('a choquei nao recebeu nenhuma punição pela postagem fake news da menina, que levou ela a se matar. saiu ilesa do pro processo')

{'classe': ['feliz', 'medo', 'nojo', 'raiva', 'triste'],
 'probabilidade': [0.00021003048980869262,
  0.0005743323818542711,
  0.5620655232159751,
  0.4348512338827211,
  0.002298880029640734]}